# Hidden Markov Model

In [3]:
import sys
sys.path.append("../../..")
import pandas as pd
import numpy as np
from pyadlml.dataset import set_data_home, fetch_uci_adl_binary
set_data_home('/home/chris/code/adlml/data_home')



subject = 'OrdonezB'
data = fetch_uci_adl_binary(cache=False, subject=subject)

/home/chris/Desktop/code/adlml/pyadlml/notebooks/models/sequential/../../../pyadlml/dataset/_core/devices.py:424: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  results = np.array(list(dask.compute(*lazy_results)))
/home/chris/Desktop/code/adlml/pyadlml/notebooks/models/sequential/../../../pyadlml/dataset/_core/devices.py:464: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  results = np.array(list(dask.compute(*dev_df_list)))


## Pipeline

In [4]:
from pyadlml.model_selection import train_test_split

print('splitting data in train and test set...')
X_train, X_test, y_train, y_test, dev_pre_vals = train_test_split(
    data.df_devices,
    data.df_activities,
    #split='leave_one_day_out', 
    split=0.7,
    return_pre_vals=True)


splitting data in train and test set...


In [3]:
from pyadlml.preprocessing import StateVectorEncoder, LabelEncoder, DropTimeIndex, CVSubset, SequenceSlicer, DfCaster
from pyadlml.pipeline import Pipeline, TrainOnlyWrapper, EvalOnlyWrapper, TrainOrEvalOnlyWrapper

steps = [
    ('encode_devices', StateVectorEncoder(encode='changepoint')),
    ('fit_labels', TrainOrEvalOnlyWrapper(LabelEncoder(idle=True))),
    ('drop_time', DropTimeIndex()),
    ('df->np', DfCaster('df->np', 'df->np')),
    ('passthrough', 'passthrough')
]

pipe = Pipeline(steps).train()

In [4]:
Xenc_train, yenc_train = pipe.fit_transform(X_train, y_train)

print('Xenc_train: ', type(Xenc_train), Xenc_train.shape, Xenc_train.dtype)
print('yenc_train: ', type(yenc_train), yenc_train.shape, yenc_train.dtype)

Xenc_train:  <class 'numpy.ndarray'> (3266, 12) float32
yenc_train:  <class 'numpy.ndarray'> (3266,) int64


## Model

In [5]:
from pomegranate import *

In [31]:
class HMM():
    def __init__(self):
        pass
    
    
    def fit(self, X, y):             
        nr_states = len(np.unique(y))
        nr_obs = X.shape[1]
        
        # create state transitions        
        trans_mat = np.ones((nr_states, nr_states))/nr_states**2
        pi = numpy.ones((nr_states,))/nr_states
        
        # create multivariate bernoulli distributions for each state
        dists = []
        for i in range(nr_states):
            b = BernoulliDistribution(0.5)
            b_dists = [BernoulliDistribution(0.5).copy() for i in range(nr_obs)]
            dists.append(IndependentComponentsDistribution(b_dists))
        
        # fit model
        self.hmm_ = HiddenMarkovModel.from_matrix(trans_mat,
            dists, starts=pi, state_names=np.unique(y).astype(str).tolist())        
        self.hmm_.fit(X, labels=y)
        
    def _ycaster(func):
        def inner(*args, **kwargs):
            return func(*args, **kwargs)
        return inner
    
    def fit_transform(self, X, y):
        y = y.astype(str)
        self.fit(X,y)
        return self.transform(X, y)
    
    def transform(self, X, y):
        self.hmm_.predict(X, algorithm='map')
    
    def predict(self, X):
        self.hmm_.predict(X, algorithm='map')
    
    def predict_proba(self, X):
        self.hmm_.predict_proba(X)

In [32]:
Xenc_train.shape, yenc_train[:10]

((3266, 12), array([ 8,  8,  8,  8, 10, 10,  8,  8,  8,  8]))

In [ ]:
hmm = HMM()
hmm.fit(Xenc_train, yenc_train)

In [ ]:
hmm.hmm_?

In [ ]:
hmm.hmm_?

In [ ]:
seq = list('CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC')

d1 = DiscreteDistribution({'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25})
d2 = DiscreteDistribution({'A': 0.10, 'C': 0.40, 'G': 0.40, 'T': 0.10})
plt.figure( figsize=(5,3))
d1.plot( n=100, edgecolor='b', facecolor='y', bins=7, alpha=0.3, label="Normal" )
d2.plot( n=100, edgecolor='b', facecolor='r', bins=10, alpha=0.3, label="second" )
plt.legend()

# General Mixture Model Setup
gmm = GeneralMixtureModel( [d1, d2] )

# Hidden Markov Model Setup
s1 = State(d1, name='background')
s2 = State(d2, name='CG Island')

hmm = HiddenMarkovModel('Activity')
hmm.add_states(s1, s2)
hmm.add_transition( hmm.start, s1, 0.5 )
hmm.add_transition( hmm.start, s2, 0.5 )
hmm.add_transition( s1, s1, 0.89 )
hmm.add_transition( s1, s2, 0.10 )
hmm.add_transition( s1, hmm.end, 0.01 )
hmm.add_transition( s2, s1, 0.1 )
hmm.add_transition( s2, s2, 0.9 )
hmm.bake()

plt.figure( figsize=(10,6) )
hmm.plot()


In [ ]:
gmm_predictions = gmm.predict( np.array(seq) )
hmm_predictions = hmm.predict( seq )


print("sequence: {}".format( ''.join( seq ) ) )
print("gmm pred: {}".format( ''.join( map( str, gmm_predictions ) ) ) )
print("hmm pred: {}".format( ''.join( map( str, hmm_predictions ) ) ) )
print("")
print("hmm state 0: {}".format( hmm.states[0].name ))
print("hmm state 1: {}".format( hmm.states[1].name ))

In [ ]:
trans, ems = hmm.forward_backward( seq )

In [ ]:
import pandas as pd
def np_arr_to_df(np_data,labels):
    return pd.DataFrame(np_data,index=labels, columns=labels)
    

In [ ]:
df_trans = np_arr_to_df(trans, ['background', 'CG-Island', 'aasd','asdf'])
print(df_trans)